É possivel dividir os decretos por NLP?

Faz sentido a clusterização?

1 - Criar request

2 - Transformar dados

3 - Gravar dados transformados para não necessitar fazer de novo

4 - Preprocessamento

5 - Vetorização

6 - Modelagem

7 - Analise

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def get_data(pg):
  res = requests.get(f'https://capivari.siscam.com.br/Documentos/Pesquisa?Pesquisa=Simples&id=80&pagina={pg}&Modulo=8&Documento=0&Numeracao=Documento&NumeroInicial=&AnoInicial=2024')
  bs = BeautifulSoup(res.content, 'html.parser')
  itens = []
  for item in bs.find_all('div', class_ = 'data-list-item'):
    itens.append(
        (
            item.find('strong', string = 'Protocolo:').parent.text.replace('\nProtocolo: ',"").replace('\r\n','').strip(),
            item.find('strong', string = 'Data:').parent.text.replace('\nData: ',"").replace('\r\n','').strip(),
            item.find('strong', string = 'Assunto:').parent.text.replace('\nAssunto: ',"").replace('\r\n','').strip(),
            item.find('strong', string = 'Situação:').parent.text.replace('\nSituação: ',"").replace('\r\n','').strip(),
            item.find('strong', string = 'Regime:').parent.text.replace('\nRegime: ',"").replace('\r\n','').strip(),
            item.find('strong', string = 'Autoria:').parent.text.replace('\nAutoria: ',"").replace('\r\n','').strip()
        )
        )
  return itens

pd.DataFrame(get_data(2), columns = ['Protocolo', 'Data', 'Assunto', 'Situação', 'Regime', 'Autoria'])

,Protocolo,Data,Assunto,Situação,Regime,Autoria
0,00342/2024,09/02/2024,Solicita reforço na pintura de solo e instalaç...,Aprovada,Ordinário,PALOMA TATIANE DA SILVA
1,00344/2024,09/02/2024,Solicita informações sobre melhorias na quadra...,Aprovada,Ordinário,PALOMA TATIANE DA SILVA
2,00341/2024,09/02/2024,Ao Excelentíssimo Prefeito Municipal de Capiva...,Aprovada,Ordinário,PALOMA TATIANE DA SILVA
3,00340/2024,08/02/2024,Solicita melhorias no asfalto da Rua Moises Bi...,Aprovada,Ordinário,PALOMA TATIANE DA SILVA
4,00332/2024,08/02/2024,Solicita criação de lei municipal para concede...,Aprovada,Ordinário,PALOMA TATIANE DA SILVA
5,00329/2024,08/02/2024,Solicita estudos para disponibilização de vaga...,Aprovada,Ordinário,PALOMA TATIANE DA SILVA
6,00328/2024,08/02/2024,Solicita que o Poder Executivo informe se exis...,Aprovada,Ordinário,PALOMA TATIANE DA SILVA
7,00327/2024,08/02/2024,Solicita a limpeza da área verde do bairro Moreto,Aprovada,Ordinário,PALOMA TATIANE DA SILVA
8,00326/2024,08/02/2024,Solicita vistoria em todas as rampas de acesso...,Aprovada,Ordinário,JEISELE RENATA ROSADA
9,00323/2024,08/02/2024,Solicita manutenção efetiva na Rua Artur Nogue...,Aprovada,Ordinário,VINÍCIUS SCARSO


In [ ]:
dt[0]

<div class="data-list-item data-list-striped data-list-hover">
<h4><a href="/Documentos/Documento/39938" target="_self" title="Indicação Nº 79/2024">Indicação Nº 79/2024</a></h4>
<div class="row">
<p class="col-md-4">
<strong>Data:</strong> 19/02/2024
        </p>
<p class="col-md-4">
<strong>Protocolo:</strong> 00580/2024
            </p>
</div>
<div class="row">
<p class="col-md-4">
<strong>Situação:</strong> Protocolada
        </p>
<p class="col-md-4">
<strong>Regime:</strong> Ordinário
            </p>
</div>
<div class="row">
<p class="col-md-12">
<strong>Autoria:</strong> THIAGO JUNIOR ANESIO BRAGGION
                </p>
</div>
<div class="row">
<p class="col-md-12">
<strong>Assunto:</strong> Solicita melhorias nas ruas do bairro Cancian
        </p>
</div>
<ul class="list-inline hidden-print">
<li>
<a class="btn btn-primary" href="/Documentos/ListarArquivosPdf/39938" target="_blank" title="Processo completo"><span aria-hidden="true" class="fa fa-file-text-o fa-lg"></span></a>


In [ ]:
dt[0].find('strong', string = 'Assunto:').parent.text

'\nAssunto: Solicita melhorias nas ruas do bairro Cancian\r\n        '